# Complete LoRA & PEFT Implementation Guide - Notebook

### What is PEFT?
* PEFT (Parameter-Efficient Fine-Tuning) is a family of techniques designed to fine-tune large language models efficiently by updating only a small subset of parameters rather than the entire model.

### Key Concepts:

* Traditional Fine-tuning: Updates all model parameters (billions of parameters)
* PEFT: Updates only a small fraction (typically <1% of parameters)
* Memory Efficient: Requires significantly less GPU memory
* Storage Efficient: Only need to save the small adapter weights

### What is LoRA?
* LoRA (Low-Rank Adaptation) is a PEFT technique that decomposes weight updates into low-rank matrices, making fine-tuning extremely efficient.

### Use Cases:

* Domain Adaptation: Adapt general models to specific domains
* Task-Specific Fine-tuning: Create specialized versions for different tasks
* Personal Assistants: Create personalized model behavior
* Resource-Constrained Environments: When full fine-tuning isn't feasible

### Step 1: Installation & Dependencies

In [ ]:
!pip install transformers torch accelerate bitsandbytes peft
!pip install pypdf2 sentence-transformers faiss-cpu
!pip install datasets

### Step 2: Core Imports Library

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Step 3: PDF Processing Pipeline

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

def chunk_text(text, chunk_size=500, overlap=50):
    """Split text into overlapping chunks"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks

### Step 4: Enhanced Vector Database

In [ ]:
class SimpleVectorDB:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.encoder = SentenceTransformer(model_name)
        self.index = None
        self.chunks = []

    def add_documents(self, texts):
        """Add text chunks to vector database"""
        self.chunks = texts
        embeddings = self.encoder.encode(texts)

        # Create FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner product similarity

        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)

    def search(self, query, k=3):
        """Search for most relevant chunks"""
        query_embedding = self.encoder.encode([query])
        faiss.normalize_L2(query_embedding)

        scores, indices = self.index.search(query_embedding, k)

        results = []
        for i, idx in enumerate(indices[0]):
            results.append({
                'text': self.chunks[idx],
                'score': scores[0][i]
            })
        return results

### Step 5: LoRA Model Setup (Your Major Addition!)

In [ ]:
def setup_model_with_lora():
    """Setup model with LoRA configuration"""

    # Quantization config for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    # Load model and tokenizer
    model_name = "microsoft/DialoGPT-medium"  # Lightweight model for Colab
    # For better results, use: "meta-llama/Llama-2-7b-chat-hf" (requires HF token)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # Add padding token if not present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Prepare model for training
    model = prepare_model_for_kbit_training(model)

    # LoRA configuration
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=16,  # Rank
        lora_alpha=32,  # LoRA scaling parameter
        lora_dropout=0.1,
        target_modules=["c_attn", "c_proj"],  # Target modules for LoRA
    )

    # Apply LoRA to model
    model = get_peft_model(model, lora_config)

    return model, tokenizer

### Step 6: Complete QA System Integration

In [ ]:
class PDFQuestionAnswering:
    def __init__(self):
        self.vector_db = SimpleVectorDB()
        self.model = None
        self.tokenizer = None

    def load_pdf(self, pdf_path):
        """Load and process PDF"""
        print("Extracting text from PDF...")
        text = extract_text_from_pdf(pdf_path)

        print("Chunking text...")
        chunks = chunk_text(text)

        print("Creating vector database...")
        self.vector_db.add_documents(chunks)

        print(f"Processed {len(chunks)} chunks from PDF")

    def setup_model(self):
        """Initialize the model with LoRA"""
        print("Setting up model with LoRA...")
        self.model, self.tokenizer = setup_model_with_lora()
        print("Model setup complete!")

    def answer_question(self, question, max_length=200):
        """Answer question using retrieved context"""
        # Retrieve relevant chunks
        relevant_chunks = self.vector_db.search(question, k=3)

        # Combine context
        context = "\n".join([chunk['text'] for chunk in relevant_chunks])

        # Create prompt
        prompt = f"""Based on the following context, answer the question:

Context: {context[:1000]}...

Question: {question}

Answer:"""

        # Tokenize and generate
        inputs = self.tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=inputs.shape[1] + max_length,
                num_return_sequences=1,
                temperature=0.7,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True
            )

        # Decode response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = response[len(prompt):].strip()

        return {
            'answer': answer,
            'context': context[:500] + "..." if len(context) > 500 else context,
            'relevant_chunks': len(relevant_chunks)
        }


### Step 7: Initialize the QA system

In [ ]:
def main():
    # Initialize the QA system
    qa_system = PDFQuestionAnswering()

    # Upload your PDF file to Colab first
    # You can use: files.upload() or mount Google Drive

    # Load your e-commerce & cybersecurity PDF
    pdf_path = "Upload you File path"

    try:
        qa_system.load_pdf(pdf_path)
        qa_system.setup_model()

        # Example questions for e-commerce & cybersecurity
        questions = [
            "What payment security protocols are discussed?"
        ]

        for question in questions:
            print(f"\nQuestion: {question}")
            result = qa_system.answer_question(question)
            print(f"Answer: {result['context']}")
            print("-" * 50)

    except FileNotFoundError:
        print("Please upload your PDF file first!")
        print("Use: from google.colab import files; files.upload()")

### Step 8: Initialize question-answering session

In [ ]:
def interactive_qa():
    """Interactive question-answering session"""
    qa_system = PDFQuestionAnswering()

    # Upload file
    from google.colab import files
    print("Please upload your PDF file:")
    uploaded = files.upload()

    # Get the uploaded file path
    pdf_path = list(uploaded.keys())[0]

    # Setup system
    qa_system.load_pdf(pdf_path)
    qa_system.setup_model()

    print("\nPDF loaded successfully! You can now ask questions.")
    print("Type 'quit' to exit.\n")

    while True:
        question = input("Your question: ")
        if question.lower() == 'quit':
            break

        result = qa_system.answer_question(question)
        print(f"\nAnswer: {result['answer']}\n")
        print("-" * 50)

### Step 9: Run and Test code

In [ ]:
if __name__ == "__main__":
    main()

Extracting text from PDF...
Chunking text...
Creating vector database...
Processed 61 chunks from PDF
Setting up model with LoRA...
Model setup complete!

Question: What payment security protocols are discussed?
Answer: Security is a critical concern for electronic payment systems (EPS) such as credit cards, debit cards, e - wallets, eChecks and smart cards, as they involve the transfer of sensitive financial information over the internet. To protect against fraud and unauthorized transactions, EPS providers employ a variety of security measures, including:  Encryption : EPS providers use encryption to protect sensitive information as it is transmitted over the internet. This makes it difficult for hackers to ...
--------------------------------------------------
